In [4]:
class DeploymentError(Exception):
    """
    Base exception for all deployment-related errors.
    Supports contextual information.
    """
    def __init__(self, message, service=None, environment=None, version=None):
        self.service = service
        self.environment = environment
        self.version = version
        super().__init__(message)

    def __str__(self):
        context = []
        if self.service:
            context.append(f"service={self.service}")
        if self.environment:
            context.append(f"env={self.environment}")
        if self.version:
            context.append(f"version={self.version}")

        context_str = ", ".join(context)
        return f"{super().__str__()} ({context_str})" if context_str else super().__str__()


class InvalidDeploymentInput(DeploymentError):
    """Raised when invalid input is provided"""
    pass


class RollbackError(DeploymentError):
    """Raised when rollback is not possible"""
    pass

class Deployment:
    """
    Manages the state and version history of a software deployment.
    """

    def __init__(self, service_name, environment):
        if not isinstance(service_name, str) or not service_name:
            raise InvalidDeploymentInput(
                "service_name must be a non-empty string",
                service=service_name,
                environment=environment
            )

        if not isinstance(environment, str) or not environment:
            raise InvalidDeploymentInput(
                "environment must be a non-empty string",
                service=service_name,
                environment=environment
            )

        self.service_name = service_name
        self.environment = environment
        self.status = "pending"
        self._history = []

    def deploy(self, new_version):
        if not isinstance(new_version, str) or not new_version:
            raise InvalidDeploymentInput(
                "new_version must be a non-empty string",
                service=self.service_name,
                environment=self.environment,
                version=new_version
            )
        self._history.append(new_version)
        self.status = "deployed"

    def rollback(self):
        if len(self._history) < 2:
            raise RollbackError(
                "Rollback not possible: no previous version",
                service=self.service_name,
                environment=self.environment
            )

        self._history.pop()
        self.status = "rolled_back"
        return True

    def check_status(self):
        return {
            "service_name": self.service_name,
            "environment": self.environment,
            "version": self._history[-1] if self._history else None,
            "status": self.status
        }

print("---- VALID DEPLOYMENT ----")
try:
    d = Deployment("payments", "prod")
    d.deploy("v1.0")
    d.deploy("v1.1")
    print(d.check_status())
except DeploymentError as e:
    print("ERROR:", e)

print("\n---- ROLLBACK ----")
try:
    d.rollback()
    print(d.check_status())
except DeploymentError as e:
    print("ERROR:", e)

print("\n---- INVALID VERSION ----")
try:
    d.deploy("")
except DeploymentError as e:
    print("ERROR:", e)

print("\n---- INVALID ENV ----")
try:
    Deployment("orders", "")
except DeploymentError as e:
    print("ERROR:", e)

print("\n---- INVALID ROLLBACK ----")
try:
    d2 = Deployment("auth", "dev")
    d2.rollback()
except DeploymentError as e:
    print("ERROR:", e)


---- VALID DEPLOYMENT ----
{'service_name': 'payments', 'environment': 'prod', 'version': 'v1.1', 'status': 'deployed'}

---- ROLLBACK ----
{'service_name': 'payments', 'environment': 'prod', 'version': 'v1.0', 'status': 'rolled_back'}

---- INVALID VERSION ----
ERROR: new_version must be a non-empty string (service=payments, env=prod)

---- INVALID ENV ----
ERROR: environment must be a non-empty string (service=orders)

---- INVALID ROLLBACK ----
ERROR: Rollback not possible: no previous version (service=auth, env=dev)


In [26]:
class InvalidEnvironmentError(Exception):
    pass

allowed_envs = ["dev", "stage", "prod"]

def validate_env(env):
    if not isinstance(env, str):
        raise InvalidEnvironmentError("Environment must be a string")

    if not env:
        raise InvalidEnvironmentError("Environment name should not be empty")

    if env not in allowed_envs:
        raise InvalidEnvironmentError(f"Invalid environment name: {env}")

    return env
  
try:
    result = validate_env("dev")
    print(f"Environment {result} validated successfully, proceeding with the deployment ")
except InvalidEnvironmentError as e:
    print(e)


Environment dev validated successfully, proceeding with the deployment 


In [35]:
class InvalidPortError(Exception):
    pass

def validate_port(port):
    if not isinstance(port, int):
        raise InvalidPortError("Port number should be an integer")

    if port not in range(1, 65536):
        raise InvalidPortError("Port number should be in the range of 1 to 65535")

    return port

try:
    result = validate_port(8080)
    print(f"Port {result} is a valid port, proceeding further")
except InvalidPortError as e:
    print(e)

Port 8080 is a valid port, proceeding further


In [38]:
class ConfigError(Exception):
    def __init__(self, message, key=None):
        self.key = key
        super().__init__(message)


    def __str__(self):
        if self.key:
            return f"{super().__str__()} (key={self.key})"
        return super().__str__()
 
def get_config_value(config, key):
    if not isinstance(config, dict):
        raise ConfigError("Config must be a dictionary")

    if key not in config:
        raise ConfigError("Missing configuration key", key=key)

    return config[key]

config = {"host": "localhost"}

try:
    get_config_value(config, "port")
except ConfigError as e:
    print(e)

config = {"host": "localhost", "port": 5432}
print(get_config_value(config, "port"))
    

Missing configuration key (key=port)
5432


In [39]:
class DeploymentError(Exception):
    """
    Base exception for all deployment-related errors.
    Supports contextual information.
    """
    def __init__(self, message, service=None, version=None):
        self.service = service
        self.version = version
        super().__init__(message)

    def __str__(self):
        context = []
        if self.service:
            context.append(f"service={self.service}")
        if self.version:
            context.append(f"version={self.version}")

        context_str = ", ".join(context)
        return f"{super().__str__()} ({context_str})" if context_str else super().__str__()


class InvalidDeploymentInput(DeploymentError):
    pass


class Deployment:
    def __init__(self, service_name):
        if not isinstance(service_name, str) or not service_name:
            raise InvalidDeploymentInput(
                "service_name must be a non-empty string",
                service=service_name
            )

        self.service_name = service_name
        self.status = "pending"
        self._history = []

    def deploy(self, new_version):
        if not isinstance(new_version, str):
            raise InvalidDeploymentInput(
                "new_version must be a string",
                service=self.service_name,
                version=new_version
            )

        if not new_version:
            raise InvalidDeploymentInput(
                "new_version must be a non-empty string",
                service=self.service_name,
                version=new_version
            )

        self._history.append(new_version)
        self.status = "deployed"

try:
    d = Deployment("payments")
    d.deploy("v1.0")
    d.deploy("")
except DeploymentError as e:
    print(e)


new_version must be a non-empty string (service=payments)


In [40]:
class ProvisionerError(Exception):
    """Base class for provisioning failures."""
    pass
 
class DiskSpaceError(ProvisionerError):
    """Raised when there is not enough disk space."""
    def __init__(self, required, available):
        super().__init__(f"Not enough disk. Required: {required}GB, Available: {available}GB")
 
class PermissionsError(ProvisionerError):
    """Raised due to file system permission issues."""
    pass
 
def provision_file(size_gb, path):
    if size_gb > 100:
        raise DiskSpaceError(required=size_gb, available=100)
    if "/root/" in path:
        raise PermissionsError(f"Cannot write to protected path: {path}")
    print("Provisioning successful.")
 
try:
    provision_file(size_gb=50, path="/root/data.bin")
except DiskSpaceError as e:
    print(f"Caught Disk Error: {e}")
except ProvisionerError as e:
    print(f"Caught Provisioner Error: {e}")
except Exception:
    print("Caught a generic exception.")

Caught Provisioner Error: Cannot write to protected path: /root/data.bin


In [41]:
class ConfigSyntaxError(ValueError):
    def __init__(self, message, line_num, text):
        full_msg = f"Syntax error on line {line_num}: {message}"
        super().__init__(full_msg)
        self.line = line_num
        self.text = text
 
def parse_config(lines):
    for i, line in enumerate(lines, 1):
        if "=" not in line:
            raise ConfigSyntaxError("Missing '=' assignment", i, line)
    return "Parsed OK"
 
config_text = ["host=server.local", "port", "timeout=30"]
 
try:
    parse_config(config_text)
except ConfigSyntaxError as e:
    print(e)
    print(f"-> Problematic text: '{e.text}'")

Syntax error on line 2: Missing '=' assignment
-> Problematic text: 'port'
